In [ ]:
using Pkg

Pkg.activate()

inlcude("generate_ivt_fields.jl")

using preprocessing

In [ ]:
"""This function loads data in given geographic bounds. It supports loading values going 'over the end' like the lon rage from 270-40 NOTE: It is expected that the longitude is given in values from 0-360 deg and lat in range from -90:90""" 
function load_data_in_geo_bounds_typed(dataset, field_id::Union{String, Symbol, Missing}, geo_bnds::GeographicBounds, indices...; precision = Float32)::Array

  lon_normal_range = geo_bnds.lon_bounds[1] < geo_bnds.lon_bounds[2]
  lat_normal_range = geo_bnds.lat_bounds[1] < geo_bnds.lat_bounds[2]

  result_dim = length(indices) + 2


  if lon_normal_range & lat_normal_range
    return dataset[field_id][geo_bnds.lon_indices[1]:geo_bnds.lon_indices[2], geo_bnds.lat_indices[1]:geo_bnds.lat_indices[2], indices...]::Array{Union{precision, Missing}, result_dim}
  elseif !lon_normal_range & lat_normal_range
    
    lon_first = dataset[field_id][geo_bnds.lon_indices[1]:end, geo_bnds.lat_indices[1]:geo_bnds.lat_indices[2],indices...]::Array{Union{precision, Missing}, result_dim}
    lon_second = dataset[field_id][1:geo_bnds.lon_indices[2], geo_bnds.lat_indices[1]:geo_bnds.lat_indices[2],indices...]::Array{Union{precision, Missing}, result_dim}
    
    return vcat(lon_first, lon_second)::Array{Union{precision, Missing}, result_dim}
  elseif lon_normal_range & !lat_normal_range
    
    lat_first = dataset[field_id][geo_bnds.lon_indices[1]:geo_bnds.lon_indices[2], geo_bnds.lat_indices[1]:end,indices...]::Array{Union{precision, Missing}, result_dim}
    lat_second = dataset[field_id][geo_bnds.lon_indices[1]:geo_bnds.lon_indices[2], 1:geo_bnds.lat_indices[2],indices...]::Array{Union{precision, Missing}, result_dim}
    
    return hcat(lat_first, lat_second)::Array{Union{precision, Missing}, result_dim}
  else
    # last case is both are over 
    lon_f_lat_f = dataset[field_id][geo_bnds.lon_indices[1]:end, geo_bnds.lat_indices[1]:end,indices...]::Array{Union{precision, Missing}, result_dim}
    lon_f_lat_s = dataset[field_id][geo_bnds.lon_indices[1]:end, 1:geo_bnds.lat_indices[2],indices...]::Array{Union{precision, Missing}, result_dim}

    lon_s_lat_f = dataset[field_id][1:geo_bnds.lon_indices[2], geo_bnds.lat_indices[1]:end,indices...]::Array{Union{precision, Missing}, result_dim}
    lon_s_lat_s = dataset[field_id][1:geo_bnds.lon_indices[2], 1:geo_bnds.lat_indices[2],indices...]::Array{Union{precision, Missing}, result_dim}

    return vcat(hcat(lon_f_lat_f, lon_f_lat_s), hcat(lon_s_lat_f, lon_s_lat_s))::Array{Union{precision, Missing}, result_dim}
  end
end

In [1]:
using NCDatasets
using DataStructures

function create_benchmark_ds(path, varname, data)
    
    vertical_size = size(data, 3)

    ps = rand(size(data, 1), size(data, 2), size(data, 4))

    ap = rand(vertical_size)

    b = rand(vertical_size)

    NCDataset(path,"c",attrib = OrderedDict("title" => "this is a test file")) do ds
        # Define the variable temperature. The dimension "lon" and "lat" with the
        # size 100 and 110 resp are implicitly created
        defVar(ds,varname,data,("lon","lat", "lev", "time"))
        defVar(ds,"ps",ps,("lon","lat", "time"))
        defVar(ds,"ap",ap,("lev",))
        defVar(ds,"b",b,("lev",))
    end
    
end

create_benchmark_ds (generic function with 1 method)

In [2]:
data = [Float32(l+i+j/k) for i = 1:60, j = 1:50, k = 1:42, l = 1:5000]

println("Data size in mem: $(sizeof(data)/1000000)")
for id in ["hus", "ua", "va"]
    path = "sample_data/benchmark_$id.nc"
    create_benchmark_ds(path, id, data)
end



Data size in mem: 2520.0


In [1]:
using BenchmarkTools
using NCDatasets


bmable = @benchmarkable  NCDataset(["sample_data/benchmark_hus.nc", "sample_data/benchmark_va.nc", "sample_data/benchmark_ua.nc"]; aggdim = "") do ds

    data_hus = ds[:hus][:, :, :, :]
    data_ua = ds[:ua][:, :, :, :]
    data_va = ds[:va][:, :, :, :]
    data_ps = ds[:ps][:, :, :]

end

tune!(bmable)

run(bmable)

BenchmarkTools.Trial: 2 samples with 1 evaluation.
 Range (min … max):  4.199 s …    4.380 s  ┊ GC (min … max): 13.76% … 17.42%
 Time  (median):     4.289 s               ┊ GC (median):    15.63%
 Time  (mean ± σ):   4.289 s ± 128.221 ms  ┊ GC (mean ± σ):  15.63% ±  2.59%

  █                                                        █  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  4.2 s          Histogram: frequency by time         4.38 s <

 Memory estimate: 7.15 GiB, allocs estimate: 3452.

In [2]:
bmable = @benchmarkable  NCDataset(["sample_data/benchmark_hus.nc", "sample_data/benchmark_va.nc", "sample_data/benchmark_ua.nc"]; aggdim = "") do ds

    data_hus = ds[:hus][1:40, 1:20, :, :]
    data_ua = ds[:ua][1:40, 1:20, :, :]
    data_va = ds[:va][1:40, 1:20, :, :]
    data_ps = ds[:ps][1:40, 1:20, :]

end samples = 15

tune!(bmable)

run(bmable)

BenchmarkTools.Trial: 3 samples with 1 evaluation.
 Range (min … max):  1.710 s …    1.932 s  ┊ GC (min … max):  6.43% … 17.18%
 Time  (median):     1.753 s               ┊ GC (median):     9.31%
 Time  (mean ± σ):   1.799 s ± 117.734 ms  ┊ GC (mean ± σ):  11.22% ±  5.56%

  █          █                                             █  
  █▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  1.71 s         Histogram: frequency by time         1.93 s <

 Memory estimate: 1.91 GiB, allocs estimate: 3452.

In [4]:
bmable = @benchmarkable  NCDataset(["sample_data/benchmark_hus.nc", "sample_data/benchmark_va.nc", "sample_data/benchmark_ua.nc"]; aggdim = "") do ds

    data_hus = ds[:hus][:, :, :, :]::Array{Float32, 4}
    data_ua = ds[:ua][:, :, :, :]::Array{Float32, 4}
    data_va = ds[:va][:, :, :, :]::Array{Float32, 4}
    data_ps = ds[:ps][:, :, :]::Array{Float64, 3}

end

tune!(bmable)

run(bmable)

BenchmarkTools.Trial: 2 samples with 1 evaluation.
 Range (min … max):  4.138 s …   4.260 s  ┊ GC (min … max): 13.98% … 18.23%
 Time  (median):     4.199 s              ┊ GC (median):    16.14%
 Time  (mean ± σ):   4.199 s ± 86.368 ms  ┊ GC (mean ± σ):  16.14% ±  3.00%

  █                                                       █  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  4.14 s         Histogram: frequency by time        4.26 s <

 Memory estimate: 7.15 GiB, allocs estimate: 3452.

In [6]:
bmable = @benchmarkable  NCDataset(["sample_data/benchmark_hus.nc", "sample_data/benchmark_va.nc", "sample_data/benchmark_ua.nc"]; aggdim = "") do ds

    data_hus = ds[:hus][1:40, 1:20, :, :]::Array{Float32, 4}
    data_ua = ds[:ua][1:40, 1:20, :, :]::Array{Float32, 4}
    data_va = ds[:va][1:40, 1:20, :, :]::Array{Float32, 4}
    data_ps = ds[:ps][1:40, 1:20, :]::Array{Float64, 3}

end

tune!(bmable)

run(bmable)

BenchmarkTools.Trial: 3 samples with 1 evaluation.
 Range (min … max):  1.725 s …    1.948 s  ┊ GC (min … max):  6.79% … 17.43%
 Time  (median):     1.786 s               ┊ GC (median):     9.80%
 Time  (mean ± σ):   1.820 s ± 115.492 ms  ┊ GC (mean ± σ):  11.57% ±  5.48%

  █              █                                         █  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  1.72 s         Histogram: frequency by time         1.95 s <

 Memory estimate: 1.91 GiB, allocs estimate: 3452.

In [12]:
bmable = @benchmarkable  NCDataset(["sample_data/benchmark_hus.nc", "sample_data/benchmark_va.nc", "sample_data/benchmark_ua.nc"]; aggdim = "") do ds
# bmable = @benchmarkable  NCDataset("sample_data/benchmark_hus.nc") do ds


    data = zeros(Float32, 60, 50, 42, 5000)

    NCDatasets.load!(variable(ds, "hus"), data, :, :, :, :)

    # for (i, id) in enumerate([:hus, :ua, :va])
    #     NCDatasets.load!(ds[id], data[i, :, :, :, :], :, :, :, :)
    # end
    # data_ps = ds[:ps][:, :, :]::Array{Float64, 3}

end

tune!(bmable)

run(bmable)

BenchmarkTools.Trial: 4 samples with 1 evaluation.
 Range (min … max):  1.417 s …    1.720 s  ┊ GC (min … max):  0.07% … 17.64%
 Time  (median):     1.644 s               ┊ GC (median):    13.94%
 Time  (mean ± σ):   1.606 s ± 131.668 ms  ┊ GC (mean ± σ):  11.87% ±  7.76%

  █                                        █   █           █  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁█▁▁▁▁▁▁▁▁▁▁▁█ ▁
  1.42 s         Histogram: frequency by time         1.72 s <

 Memory estimate: 2.35 GiB, allocs estimate: 61.